# This is a simple notebook showing how to use the universal sentence encoder with some basic algorithms like SVC, Ridge Regression, Gaussian Kernel and some more. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Base libraries 

In [ ]:
#This is requiered to run Tensroflow HUB
!pip install -q tensorflow-text


In [ ]:
import numpy as np 
import pandas as pd 
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import time
import numpy as np
import tensorflow_hub as hub
import tensorflow_text

# Process

**We will use the raw data, without preprocessing (without cleaning). For this project, I have noticed that cleaning the data does not greatly improve model accuracy, like 1% or less, and in this case we will use simple machine learning models. I did the experiment and after spending hours cleaning the data and doing a super complex deep learning model, I was able to improve the score by only 2%, then to be able to get 80% accuracy without cleaning and with simple models, I think it's a good achievement**

There is a good notebook that perfoms an incredible cleaning procees and also explain the visualization of the data
https://www.kaggle.com/gunesevitan/nlp-with-disaster-tweets-eda-cleaning-and-bert

### We will be using the universal sentence encoder large (USE), the 5th version, so lets initiate the connection

In [ ]:
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

### Get the data and preprare de function for the USE

In [ ]:
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [ ]:
# Function to embeed the tweets with the Universal Sentence Encoder
def embedd(dataset):
    data = []
    for i in tqdm(dataset):
      embeddings = use(i)
      embeddings = tf.reshape(embeddings, [-1]).numpy()
      data.append(embeddings)
    return data

In [ ]:
#First we need to reshape our data, the version 4th of the USE worked like use("sentence to embeed") but in the 5th version you need to pass a list as 
# a parameter like: use(["sentence to embeed"]), for that reason we are reshaping our data
x_training = np.reshape(train_df.text.values,(len(train_df.text.values),1))
x_topredict = np.reshape(test_df.text.values,(len(test_df.text.values),1))

print("Embedding training data...")
x_training = embedd(x_training)
print("Embedding data for prediction...")
x_topredict = embedd(x_topredict)

In [ ]:
x_training = np.array(x_training)
y_train = train_df.target.values
x_topredict = np.array(x_topredict)

### Splitting the data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_training, y_train,test_size = 0.25, random_state=7)

## Lets try first with the RidgeRegression algorith

In [ ]:
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
clf = linear_model.RidgeClassifier()
clf.fit(X_train, y_train)

#### Create a confusion matrix to test the data

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("The accuracy of the model in the tested data is: ",accuracy_score(y_test, y_pred))
print("The f1 score of the model in the tested data is: ",f1_score(y_test, y_pred))

#### As we can see we get an accuracy of 0.8156 and f1 of 0.77, this is a really good score for a simple linear model and without cleaning the data

# Now lets try with some ML algorithms from the Scikit Learn librarie

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

log_clf.fit(X_train, y_train)
rnd_clf.fit(X_train, y_train)
svm_clf.fit(X_train, y_train)

In [ ]:
"""###Logstic regression###"""
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
y_pred = log_clf.predict(X_test)
y_pred = (y_pred > 0.5)
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("The accuracy of the model Logistic regression in the tested data is: ",accuracy_score(y_test, y_pred))
print("The f1 score of the model Logistic regression in the tested data is: ",f1_score(y_test, y_pred))


"""###Random Forest###"""
y_pred = rnd_clf.predict(X_test)
y_pred = (y_pred > 0.5)
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("The accuracy of the model Random Forest in the tested data is: ",accuracy_score(y_test, y_pred))
print("The f1 score of the model Random Forest in the tested data is: ",f1_score(y_test, y_pred))

"""###SVC kernel###"""
y_pred = svm_clf.predict(X_test)
y_pred = (y_pred > 0.5)
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("The accuracy of the model SVC in the tested data is: ",accuracy_score(y_test, y_pred))
print("The f1 score of the model SVC in the tested data is: ",f1_score(y_test, y_pred))

#### As we can see the SVC model was the one that performed better, so lets submitt that one to see our score

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
y_pred = svm_clf.predict(x_topredict)
y_pred = (y_pred > 0.5).astype(int)
sample_submission["target"] = y_pred
sample_submission.to_csv("submission_SVC_universal_sentence_encoder.csv", index=False)